In [81]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

rng = np.random.default_rng(0)

In [295]:
num_sites = 2

tunneling_strength = 1
interaction_strength = 10
chemical_potential = 0

imaginary_time_interval_duration = 1/5

temperature = 1/2
num_imaginary_time_intervals = 10
imaginary_time_interval_duration = (1/temperature ) / num_imaginary_time_intervals

stratonovich_interaction_strength = np.arccosh(np.exp((interaction_strength * imaginary_time_interval_duration)/2))

In [296]:
def compute_tunneling():
    tunneling = np.zeros((num_sites, num_sites))

    tunneling += np.diag(np.ones(num_sites-1), k=-1)
    tunneling += np.diag(np.ones(num_sites-1), k=+1)

    tunneling[num_sites-1, 0] += 1
    tunneling[0, num_sites-1] += 1

    return -tunneling_strength * tunneling

In [297]:
def compute_density():
    return -chemical_potential * np.diag(np.ones(num_sites))

In [298]:
def compute_kinetic(tunneling, density):
    return imaginary_time_interval_duration * (tunneling + density)

In [299]:
def compute_potential(field):
    potential_up = stratonovich_interaction_strength * tf.linalg.diag(field.T)
    potential_down = -potential_up

    return np.array([potential_up, potential_down])

In [300]:
def compute_propagator(potential):
    propagator = np.zeros((2, num_sites, num_sites))

    for spin in range(2):
        product = np.eye(num_sites)
        for imaginary_time_interval in range(num_imaginary_time_intervals-1, 0, -1):
            product = tf.linalg.expm(kinetic) @ tf.linalg.expm(potential[spin][imaginary_time_interval]) @ product
        propagator[spin] = tf.linalg.inv(np.eye(num_sites) + product)

    return propagator

In [301]:
def plot_propagator(propagator):
    figure, axes = plt.subplots(1, 2)

    axes[0].imshow(propagator[0])
    axes[0].set_title(r"$\downarrow$")

    axes[1].imshow(propagator[1])
    axes[1].set_title(r"$\uparrow$")
    
    figure.show()

In [302]:
kinetic = imaginary_time_interval_duration * (compute_tunneling() + compute_density())

kinetic

array([[ 0. , -0.4],
       [-0.4,  0. ]])

In [303]:
field = rng.choice([-1.0, +1.0], size=(num_sites, num_imaginary_time_intervals))
potential = compute_potential(field)
propagator = compute_propagator(potential)

propagator

array([[[0.94698221, 0.88496795],
        [0.05120892, 0.04786264]],

       [[0.05301779, 0.05120892],
        [0.88496795, 0.95213736]]])

In [304]:
for imaginary_time_interval in range(num_imaginary_time_intervals - 1, 0, -1):
    for site in range(num_sites):
        acceptance_ratio = ((1 + (1 - propagator[0][site, site]) * (np.exp(-2 * stratonovich_interaction_strength * field[site, imaginary_time_interval]) - 1)) * 
                            (1 + (1 - propagator[1][site, site]) * (np.exp(+2 * stratonovich_interaction_strength * field[site, imaginary_time_interval]) - 1)))

        if rng.random() < acceptance_ratio: 
            field[site, imaginary_time_interval] *= -1
            potential = compute_potential(field)
            propagator = compute_propagator(potential)

    for spin in range(2):
        propagator[spin] = (tf.linalg.expm(kinetic) @ tf.linalg.expm(potential[spin, imaginary_time_interval])) @ propagator[spin] @ tf.linalg.inv(tf.linalg.expm(kinetic) @ tf.linalg.expm(potential[spin, imaginary_time_interval]))

In [305]:
propagator

array([[[0.81303938, 0.44915562],
        [0.07338209, 0.8237061 ]],

       [[0.18696062, 0.07338209],
        [0.44915562, 0.1762939 ]]])

In [306]:
occupation = 1 - np.einsum('sii -> si', propagator)
occupation

array([[0.18696062, 0.1762939 ],
       [0.81303938, 0.8237061 ]])

In [307]:
doubles_occupancy = np.einsum('i, i -> i', (1 - np.diag(propagator[0])), (1 - np.diag(propagator[1])))
doubles_occupancy

array([0.15200634, 0.14521436])